In [17]:
# !pip install bs4
# !pip install webdriver_manager
# !pip install selenium==3.141.0


In [18]:
# selenium 세팅 (1)
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(ChromeDriverManager().install())

# driver.get('https://www.naver.com')

In [19]:
# selenium 세팅 (2)

from selenium.webdriver.support.ui import WebDriverWait as wait #페이지가 로딩 다 되면 동작을 시작하게끔 해줌
from selenium.webdriver.support import expected_conditions as EC  #예외적인 상황에서 에러가 떴을 때 넘겨주는 코드
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By
# 추가적으로 크롬드라이버 돌릴 때 몇가지 세팅값들을 넣으려고??? 뭔 말이지 

In [20]:
import time
import csv
import pandas as pd
import random
from bs4 import BeautifulSoup as bs

# 네이버 로그인

자동화된 크롬창을 실행시켜서 네이버 로그인 페이지 접속

1. 크롬드라이버 제어 (크롬창 켜기)
2. 네이버 로그인 (아이디 입력 -> 비밀번호 입력 -> 로그인 버튼 클릭 순서로 작업 진행)

In [21]:
# 크롬드라이버 제어

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dex-shm-usage')

# 현재 컴퓨터 크롬 드라이버 위치 파악
# chrome_path = r'chromedriver.exe' ## 본인의 크롬 드라이버 위치 입력  <- 요거는
# ChromeDriverManager().install()   # <- 요거는 새 install


# 자동화된 크롬 창 실행
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
# driver.get('https://www.naver.com')

C:\Users\user\AppData\Local\Temp\ipykernel_6772\2459345217.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


In [22]:
# 네이버 로그인 페이지 접속
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
driver.implicitly_wait(10)

#본인의 아이디, 비밀번호를 각 변수에 저장
my_id='scv0206'
my_pw='DlDpWl026!@'

# 아이디, 비밀번호 입력 (네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 사용)
#네이버에서 send_key()를 쓰면 로그인에서 자동입력방지기능이 나타남.
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

# 로그인 버튼 클릭
driver.find_element('id', 'log.login').click()
time.sleep(1)

그중 가장 많이 제시했던 방법은 execute_script('js작성가능') 메서드를 사용하는 것이 었는데, js코드를 인자로 넣을 수 있어 브라우저 화면을 제어 할수 있게 된다. 스크롤을 하기 위해 뿐만아니라 다른 동적이벤트를 작성할 수 있다.

해당 페이지에 스크립트를 만들 때 사용합니다. 요소는 필수 파라미터는 아니고 요소가 있으면 요소에 스크립트가 실행되고 없으면 전체 페이지에 스크립트가 움직입니다.

화면상 스크롤 위치 이동 : scrollTo(x,Y) ,scrollTo(x,Y+number)
화면 최하단으로 스크롤 이동 : scrollTo(0, document.body.scrollHeight)
화면을 움직이고 페이지 로드 기다리기 : time.sleep(second)

## 크롤링 할 카페 접속 및 원하는 게시판 클릭

In [23]:
# 카페 접속
time.sleep(2)
moms_url = 'https://cafe.naver.com/sqlpd' # 타겟 카페가 바뀔 경우 url을 바꿔주세요
driver.get(moms_url)
time.sleep(1)

# 원하는 게시판 클릭
driver.find_element('xpath','//*[@id="menuLink120"]').click() #게시판의 선택자 넣기
time.sleep(1)

# 프레임 전환
driver.switch_to.frame('cafe_main')
time.sleep(1)


## 게시물 크롤링

1. 첫번째 게시물 클릭 - XPath 활용
2. 크롤링할 개수 지정
3. 크롤링


In [24]:
# 제목, 본문, 댓글, 시간 빈 list 생성

titles = [] # 제목
reviews = [] # 리뷰
comments = [] # 댓글
date = [] # 시간

count_view = 0

# 1. 첫번째 게시물 클릭 (선택자 입력: XPath)
driver.find_element('xpath','//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a').click()

# 2. 크롤링 (for문으로 지정한 게시물 개수까지 돌림)
for i in range(1): # 새로 창 열기
    for i in range(30): #한 번에 몇 개의 게시물을 크롤링할 것?
        driver.implicitly_wait(30)
        
        # 제목 수집
        time.sleep(random.uniform(0.5, 1))  
        title = driver.find_element('xpath', '//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3').text
        titles.append(title)
        
        # 본문 수집 (3가지 경우 존재)
        try:
            content = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text').text
            reviews.append(content)
            
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR, 'div.ContentRenderer').text
                reviews.append(content)
            except:
                content = [] # 본문이 사진으로만 구성 (담을게 없어서 빈 리스트 생성해서 append 시켜줌)
                reviews.append(content)
        
        # 댓글 수집 (댓글이 없는  경우, 있는 경우 2가지 존재)
        soup = bs(driver.page_source, 'lxml') # 현재 페이지 HTML 정보 가지고 오기
        iscomment = soup.find_all('span', class_='text_comment')
        if len(iscomment) == 0: # 댓글 없는 경우
            box = []
        else: # 댓글 있는 경우
            wait(driver,15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'text_comment')))
            # 웹 드라이버를 통해 브라우저에서 최대 15초 기다려주고, text_comment(댓글 선택)
            
            soup = bs(driver.page_source, 'lxml')
            lscomment = soup.find_all('span', class_='text_comment')
            box = []
            for i in iscomment:
                box.append([i.get_text()])
                
        comments.append(box)
        
        
        # 날짜(시간) 수집
        times = driver.find_element('xpath', '//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]').text
        date.append(times)
        
        
        # count_view 크롤링 진행 상황 (50개마다 출력)
        count_view = count_view + 1
        if count_view % 50 == 0: # 50으로 나누었을 때 0이 되면 게시물 크롤링 완료라고 뜨도록 설정
            print('{}개 게시물 크롤링 완료'.format(count_view))
        else:
            pass
        
        # 다음 글 클릭(다음글 버튼 클릭해서 연속적으로 크롤링)
        try:
            driver.find_element(By.CSS_SELECTOR, "#app > div > div > div.ArticleTopBtns > div.right_area > a.BaseButton.btn_next.BaseButton--skinGray.size_default > span").click()
            driver.implicitly_wait(25)
        except: # 다음글 버튼 사라지는 문제점 발생, 크롬 close 후 다시 open
            current_url = driver.current_url # 다음글 버튼이 사라져서 크롤링이 멈춘 url 가지고 오기
            driver.close()
            
            # 크롬창 다시 열기
            driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
                    

            # 재 로그인
            ## 네이버 화면 이동
            login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
            driver.get(login_url)
            driver.implicitly_wait(9)

            driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
            driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
            time.sleep(1)

            # 로그인 버튼 클릭
            driver.find_element('id', 'log.login').click()
            time.sleep(1)

            # 멈춘 부분의 링크 가지고 오기
            driver.get(current_url)
            driver.switch_to.frame('cafe_main')
            time.sleep(4)
    
    # 크롬이 다시 열릴 때 가지고 올 다음 링크 가져오기 #다음글 클릭에 해당되는 href 추출 (url로 이동해야 하기 때문에)
    link = driver.find_element('xpath','//*[@id="app"]/div/div/div[1]/div[2]/a[2]').get_attribute("href")

    # 현재 크롬창 닫기
    driver.close()

    # 크롬창 다시 열기
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
    
    # 재 로그인
    ## 네이버 화면 이동
    login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
    driver.get(login_url)
    driver.implicitly_wait(9)

    driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
    driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
    time.sleep(1)

    # 로그인 버튼 클릭
    driver.find_element('id', 'log.login').click()
    time.sleep(1)

    # 다음 링크로부터 가져오기
    driver.get(link)
    driver.switch_to.frame('cafe_main')

# 크롬 창 닫기
driver.quit()

        
    

NameError: name 'chrome_path' is not defined

In [ ]:
titles

: 

In [ ]:
#데이터 합치기(크롤링한 데이터를 하나의 dataframe형태로 바꿔줌)
data={'time':date,
      'tit':titles,
      'review':reviews,
      'comment': comments}
dataDF=pd.DataFrame(data)

: 

In [ ]:
dataDF

In [ ]:
#CSV로 저장
dataDF.to_csv("데이터분석카페.csv", encodings='utf-8-sis')#저장할 파일명.csv입력후 실행